In [1]:
from pathlib import Path
import logging
import importlib

import rospy
from scipy.spatial.transform import Rotation as R

from alr_sim.core import Scene

from grasp_benchmark.utils.ycb_utils import YCBLoader
from alr_simulation_tools.scene_utils import execute_grasping_sequence, create_sample_data, reset_scene

from grasping_benchmarks_ros.srv import (
    GraspPlanner,
    GraspPlannerResponse,
)

from grasp_benchmark.utils.ros_utils import (
    create_grasp_planner_request,
    calc_point_cloud_from_images
) 

pybullet build time: Oct 28 2022 16:12:21


[Warning] Since Open3D 0.15, installing Open3D via conda is deprecated. Please re-install Open3D via: `pip install open3d -U`.
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# define scene parameters

factory_string = "mj_beta"
object_pos = (0.5, 0.0, 0.2)
adhust_object_position = True
cam_pos = (0.5, 0.0, 1)
cam_quat = [0.7071067811865476, 0.0, 0.0, -0.7071067811865475]
# cam_quat = R.from_euler("xyz", (0, 0, -90), degrees=True).as_quat()[
#         [3, 0, 1, 2]
#     ].tolist(),
# cam_quat = R.from_euler("zyx", (-90, -30, 0), degrees=True).as_quat()[
#         [3, 0, 1, 2]
#     ].tolist(),
# cam_quat = (
#     R.from_euler("zyx", (90, 50, 0), degrees=True).as_quat()[[3, 0, 1, 2]].tolist()
# )
cam_height = 480
cam_width = 640
robot_pos = (0.0, 0.5, 0.2)
robot_quat = (0, 1, 0, 0)
render_mode = Scene.RenderMode.HUMAN    
wait_time = 0.5
move_duration = 2

#
grasp_planner_service_id = "grconvnet_bench/grconvnet_grasp_planner_service"
# grasp_planner_service_id = "ggcnn_bench/ggcnn_grasp_planner_service"


In [3]:
ycb_loader = YCBLoader(
    ycb_base_folder=Path.home() / "Documents" / "SF-ObjectDataset" / "YCB", factory_string="mj_beta"
)
obj_ids = [p.name for p in ycb_loader.ycb_base_folder.iterdir() if p.is_dir()]

In [4]:
rospy.init_node("grasp_planner_client")

importlib.reload(logging)
logging.basicConfig(level=logging.INFO)

In [5]:
logging.info("Starting")
for obj_id in obj_ids:

    logging.info(f"Creating sample data for {obj_id}")

    sim_obj = ycb_loader.get_ycb_object(pos = (0.5, 0.0, 0.2), quat = (0,1,0,0), obj_id=obj_id, name=obj_id)

    results, scene, agent = create_sample_data(
        factory_string=factory_string,
        cam_pos=cam_pos,
        cam_quat=cam_quat,
        cam_height=cam_height,
        cam_width=cam_width,
        robot_pos=robot_pos,
        robot_quat=robot_quat,
        object_list=[sim_obj],
        target_obj_name=obj_id,
        render_mode=render_mode,
        wait_time=wait_time,
        move_duration=move_duration,
    )
    logging.info("Sample data created")

    pc_points, pc_colors = calc_point_cloud_from_images(
        rgb_img=results["rgb_img"], depth_img=results["depth_img"], cam_intrinsics=results["cam_intrinsics"]
    )

    grasp_req = create_grasp_planner_request(
        rgb_img=results["rgb_img"],
        depth_img=results["depth_img"],
        seg_img=results["seg_img"],
        pc_points=pc_points,
        pc_colors=pc_colors,
        cam_pos=results["cam_pos"],
        cam_quat=results["cam_quat"],
        cam_intrinsics=results["cam_intrinsics"],
        cam_height=cam_height,
        cam_width=cam_width,
        num_of_candidates=10_000,
    )

    logging.info("Created request, sending request ...")

    rospy.wait_for_service(grasp_planner_service_id, timeout=30.0)
    grasp_planner = rospy.ServiceProxy(grasp_planner_service_id, GraspPlanner)

    reply: GraspPlannerResponse = grasp_planner(grasp_req)
    logging.info("Received grasp candidates")

    best_grasp = sorted(reply.grasp_candidates, key=lambda x: x.score.data)[-1]
    grasp_pos = best_grasp.pose.pose.position
    grasp_pos = (grasp_pos.x, grasp_pos.y, grasp_pos.z)
    grasp_quat = best_grasp.pose.pose.orientation
    grasp_quat = (grasp_quat.w, grasp_quat.x, grasp_quat.y, grasp_quat.z)

    execute_grasping_sequence(
        agent = agent,
        grasp_pos = grasp_pos,
        grasp_quat = grasp_quat
    )

    reset_scene(factory_string, scene, agent)
        

INFO:root:Starting
INFO:root:Creating sample data for 002_master_chef_can
INFO:root:Sample data created
INFO:root:Created request, sending request ...
INFO:root:Received grasp candidates
INFO:root:Going to home position (0.5, 0, 0.5)
INFO:root:Going to hover_ position [0.4902142  0.03706899 0.14799997]
INFO:root:Opening gripper
INFO:root:Going to grasp position (0.4902142042662144, 0.03706899390115767, 0.09799997091293379)
INFO:root:Closing gripper
INFO:root:Going to hover position [0.4902142  0.03706899 0.14799997]
INFO:root:Going to home position (0.5, 0, 0.5)
INFO:root:Going to drop position (0, 0.5, 0.5)
INFO:root:Opening gripper
INFO:root:Closing gripper
INFO:root:Creating sample data for 002_master_chef_can_orig
INFO:root:Sample data created
INFO:root:Created request, sending request ...
INFO:root:Received grasp candidates
INFO:root:Going to home position (0.5, 0, 0.5)
INFO:root:Going to hover_ position [0.4902142  0.03706899 0.14799997]
INFO:root:Opening gripper
INFO:root:Going 

KeyboardInterrupt: 